# Denoising Diffusion Probabilistic Model

デノイジング拡散確率モデル（Denoising Diffusion Probabilistic Model, DDPM）とは、ノイズが付与されたデータを潜在変数と捉え、元データに対しノイズを付与していく過程（拡散過程）を元に、潜在変数から元データを復元する過程（逆拡散過程、生成過程）を通じてデータを生成することを目指す、潜在変数モデルの一種である。

## 潜在変数モデル

潜在変数モデルは、観測変数$x$を生成する確率分布の裏に、何らかの確率的潜在変数$z$の存在を仮定するようなモデルである。
$$
p(x) = \int_{\mathcal{Z}} p(x|z) p(z) dz
$$

潜在変数モデルにおける学習はMAP推定とベイズ推定の二種類存在するが、後者のベイズ推定では、通常事後分布$p(z|x)$を目標とした学習を行う；
潜在変数モデルの分類器（この場合変分分布と呼んでも良い）を、$\theta$をパラメータとして$\hat{p}_\theta(z)$とすると、$\hat{p}_\theta(z)$と$p(z|x)$のKLダイバージェンスを最小化することパラメータの推定値$\theta^{\text{LVM}}$を得る。
$$
\begin{align*}
\theta^{\text{LVM}}
 &= \argmin_{\theta} \text{KL} \left( \hat{p}_\theta(z) || p(z|x) \right) \\
 &= \argmin_{\theta} \int_{\mathcal{Z}} \hat{p}_\theta(z) \ln{\left( \frac{\hat{p}_\theta(z)}{p(z|x)} \right)} dz \\
 &= \argmin_{\theta} \left( \int_{\mathcal{Z}} \hat{p}_\theta(z) \ln{\left( \frac{p(x, z)}{p(z|x)} \right)} dz - \int_{\mathcal{Z}} \hat{p}_\theta(z) \ln{\left( \frac{p(x, z)}{\hat{p}_\theta(z)} \right)} dz \right) \\
 &= \argmin_{\theta} \left( \int_{\mathcal{Z}} \hat{p}_\theta(z) \ln{p(x)} dz - \int_{\mathcal{Z}} \hat{p}_\theta(z) \ln{\left( \frac{p(x, z)}{\hat{p}_\theta(z)} \right)} dz \right) \\
 &= \argmin_{\theta} \left( \ln{p(x)} - \int_{\mathcal{Z}} \hat{p}_\theta(z) \ln{\left( \frac{p(x, z)}{\hat{p}_\theta(z)} \right)} dz \right) \\
 &= \argmin_{\theta} \left( \ln{p(x)} - \mathbb{E}_{z \sim \hat{p}_\theta(z)} \left[ \ln{\left( \frac{p(x, z)}{\hat{p}_\theta(z)} \right)} \right] \right)
\end{align*}
$$
ここで、最右辺第一項$\ln{p(x)}$が$\theta$に依存せず無視できるため、KLダイバージェンスの最小化は第二項の最大化と等価であると言える。
$$
\theta^{\text{LVM}} = \argmax_{\theta} \mathbb{E}_{z \sim \hat{p}_\theta(z)} \left[ \ln{\left( \frac{p(x, z)}{\hat{p}_\theta(z)} \right)} \right]
$$
この項を変分下限（Evidence Lower Bound, ELBO）と呼ぶ。
「変分下限」という名称は、KLダイバージェンスが常に正の値をとることにより成立する不等式：
$$
\begin{align*}
\ln{p(x)} \geq \mathbb{E}_{z \sim \hat{p}_\theta(z)} \left[ \ln{\left( \frac{p(x, z)}{\hat{p}_\theta(z)} \right)} \right]
\end{align*}
$$
に由来する。

### Example 1. ディリクレ-多項分布モデル

ディリクレ-多項分布モデルでは、潜在変数$z_1, \cdots, z_K$がディリクレ分布から生起し、これらをパラメータとした多項分布によって$x$が生起する；
$$
\begin{align*}
p(z_1, \cdots, z_K; \alpha_1, \cdots, \alpha_K) &= \frac{\Gamma \left( \sum_{k=1}^K \alpha_k \right)}{\prod_{k=1}^K \Gamma \left( \alpha_k \right)} \prod_{k=1}^K z_k^{\alpha_k-1} \\
p(x_1, \cdots, x_K| z_1, \cdots, z_K; n) &= \frac{n!}{\prod_{k=1}^K x_k!} \prod_{k=1}^K z_k^{x_k}
\end{align*}
$$
ただし、$x_1, \cdots, x_K$について$\sum_{k=1}^K x_k = n$と制約される。
同時分布、および事後分布は、簡単な計算によって解析的に得られる。
$$
\begin{align*}
p(x, z; \alpha_1, \cdots, \alpha_K, n)
 &= \frac{n!}{\prod_{k=1}^K x_k!} \frac{\Gamma \left( \sum_{k=1}^K \alpha_k \right)}{\prod_{k=1}^K \Gamma \left( \alpha_k \right)} \prod_{k=1}^K z_k^{x_k+\alpha_k-1} \\
p(z|x; \alpha_1, \cdots, \alpha_K, n)
 &= \frac{\Gamma \left( \sum_{k=1}^K (x_k + \alpha_k) \right)}{\prod_{k=1}^K \Gamma \left( x_k +\alpha_k \right)} \prod_{k=1}^K z_k^{x_k+\alpha_k-1}
\end{align*}
$$

さて、変分下限の最大化によって、同じ事後分布が得られることを確認する。今、変分分布$q_\theta(z)$を、パラメータを$\gamma$とするディリクレ分布によって定義すると、変分下限は
$$
\begin{align*}
\mathbb{E}_{z \sim \hat{p}_\theta(z)} \left[ \ln{\left( \frac{p(x, z)}{\hat{p}_\theta(z)} \right)} \right]
 &= \mathbb{E}_{z \sim \hat{p}_\theta(z)} \left[ \ln{\left( \frac{n!}{\prod_{k=1}^K x_k!} \frac{\Gamma \left( \sum_{k=1}^K \alpha_k \right)}{\prod_{k=1}^K \Gamma \left( \alpha_k \right)} \prod_{k=1}^K z_k^{x_k+\alpha_k-1} \right)} - \ln{\left( \frac{\Gamma \left( \sum_{k=1}^K \gamma_k \right)}{\prod_{k=1}^K \Gamma \left( \gamma_k \right)} \prod_{k=1}^K z_k^{\gamma_k - 1} \right)} \right] \\
 &= \mathbb{E}_{z \sim \hat{p}_\theta(z)} \left[ \sum_{k=1}^K (x_k + \alpha_k - \gamma_k) \ln{z_k} - \ln{\left( \frac{\Gamma \left( \sum_{k=1}^K \gamma_k \right)}{\prod_{k=1}^K \Gamma \left( \gamma_k \right)} \right)} \right] + \text{Const.}\\
 &= \sum_{k=1}^K (x_k + \alpha_k - \gamma_k) \mathbb{E}_{z \sim \hat{p}_\theta(z)} \left[ \ln{z_k} \right] - \ln{\Gamma \left( \sum_{k=1}^K \gamma_k \right)} + \sum_{k=1}^K  \ln{\Gamma \left( \gamma_k \right)} + \text{Const.}\\
\end{align*}
$$
ここで、ディリクレ分布の性質として、$\varphi(\gamma)$をディガンマ関数とすると、
$$
\mathbb{E}_{z \sim \hat{p}_\theta(z)} \left[ \ln{z_k} \right] = \varphi{\left( \gamma_k \right)} - \varphi{\left( \sum_{i=1}^K \gamma_i \right)}
$$
が成立する[[Source](https://en.wikipedia.org/wiki/Dirichlet_distribution)]ため、最終的に変分下限は以下のように導かれる。
$$
\mathbb{E}_{z \sim \hat{p}_\theta(z)} \left[ \ln{\left( \frac{p(x, z)}{\hat{p}_\theta(z)} \right)} \right] = \sum_{k=1}^K (x_k + \alpha_k - \gamma_k) \left( \varphi{\left( \gamma_k \right)} - \varphi{\left( \sum_{i=1}^K \gamma_i \right)} \right) - \ln{\Gamma \left( \sum_{k=1}^K \gamma_k \right)} + \sum_{k=1}^K  \ln{\Gamma \left( \gamma_k \right)} + \text{Const.}\\
$$

さて、各項に対して$\gamma_j$で微分することを考える。まず第一項について、$k=j$の時とそうでない時で場合分けると、
$$
\begin{align*}
\frac{\partial}{\partial \gamma_j} \sum_{k=1}^K (x_k + \alpha_k - \gamma_k) \left( \varphi{\left( \gamma_k \right)} - \varphi{\left( \sum_{i=1}^K \gamma_i \right)} \right)
 &= \frac{\partial}{\partial \gamma_j} (x_j + \alpha_j - \gamma_j) \left( \varphi{\left( \gamma_j \right)} - \varphi{\left( \sum_{i=1}^K \gamma_i \right)} \right) + \frac{\partial}{\partial \gamma_j} \sum_{k=1, \cdots, K: k \neq j} (x_k + \alpha_k - \gamma_k) \left( \varphi{\left( \gamma_k \right)} - \varphi{\left( \sum_{i=1}^K \gamma_i \right)} \right) \\
 &= -\left( \varphi{\left( \gamma_j \right)} - \varphi{\left( \sum_{i=1}^K \gamma_i \right)} \right) + (x_j + \alpha_j - \gamma_j) \left( \varphi^{(1)}{\left( \gamma_j \right)} - \varphi^{(1)}{\left( \sum_{i=1}^K \gamma_i \right)} \right) + \sum_{k=1, \cdots, K: k \neq j} (x_k + \alpha_k - \gamma_k) \left(- \varphi^{(1)}{\left( \sum_{i=1}^K \gamma_i \right)} \right) \\
 &= - \varphi{\left( \gamma_j \right)} + \varphi{\left( \sum_{i=1}^K \gamma_i \right)} + (x_j + \alpha_j - \gamma_j) \varphi^{(1)}{\left( \gamma_j \right)} -  \sum_{k=1}^K (x_k + \alpha_k - \gamma_k) \varphi^{(1)}{\left( \sum_{i=1}^K \gamma_i \right)}
\end{align*}
$$
と書ける（$\varphi^{(1)}(\gamma)$はトリガンマ関数）。
一方で第二項、第三項は容易に、
$$
\frac{\partial}{\partial \gamma_j} \left( - \ln{\Gamma \left( \sum_{k=1}^K \gamma_k \right)} + \sum_{k=1}^K  \ln{\Gamma \left( \gamma_k \right)} \right)
 = - \varphi{\left( \sum_{k=1}^K \gamma_k \right)} + \varphi{\left( \gamma_j \right)}
$$
で得られるため、これらを合わせることで、変分下限の微分が得られる：
$$
\frac{\partial}{\partial \gamma_j} \mathbb{E}_{z \sim \hat{p}_\theta(z)} \left[ \ln{\left( \frac{p(x, z)}{\hat{p}_\theta(z)} \right)} \right] = (x_j + \alpha_j - \gamma_j) \varphi^{(1)}{\left( \gamma_j \right)} -  \sum_{k=1}^K (x_k + \alpha_k - \gamma_k) \varphi^{(1)}{\left( \sum_{i=1}^K \gamma_i \right)}
$$
変分下限の微分を$0$とおいた時の解について考えると、KLダイバージェンスが狭義凸であるため、変分下限の停留点が高々一つしか存在し得ないことに注意すれば、自明解：
$$
\forall j, \quad \gamma_j = x_j + \alpha_j
$$
が最大の変分下限を実現すると言える。
以上より、変分下限の最大化によって得られる解が、真の事後分布と同じ確率分布を表現することが示された。

## DDPMの概要

DDPMは、観測変数$x=x_0$の裏に、$T$個の潜在変数$x_{1:T} = x_1, \cdots, x_T$が一列に並んで接続されたモデルである。
> グラフィカルモデル：$x_T \rightarrow x_{T-1} \rightarrow \cdots \rightarrow x_2 \rightarrow x_1 \rightarrow x_0$

観測変数$x_0$から潜在変数$x_{1:T}$を順に得るプロセス$q(x_{1:T}|x_0)$を拡散過程と呼ぶ。
一般に分類器と呼ばれるこの分布に「拡散過程」という名前が付けられているのは、後に示すように、元のデータ$x_0$に対しノイズを加える（同時にデータ成分を減衰させる）操作を繰り返すことによって、データが正規分布に従うように「拡散」すること、またそのプロセスがマルコフ過程の一種と認められることに由来する。
一方で、一番根源的な潜在変数$x_T$から、その他の潜在変数$x_{1:T-1}$を逆順にたどり、最終的に観測変数$x_0$を得るプロセス$p(x_0, x_{1:T})$を逆拡散過程、または生成過程と呼ぶ。
こちらも一般には生成器などと呼ばれる。
具体的な定式化は後述する。
> 拡散過程：$x_0 \rightarrow x_{1} \rightarrow \cdots \rightarrow x_{T-2} \rightarrow x_{T-1} \rightarrow x_T$

> 生成過程：$x_T \rightarrow x_{T-1} \rightarrow \cdots \rightarrow x_2 \rightarrow x_1 \rightarrow x_0$

一般的な潜在変数モデルは、上で挙げた例のように、まず生成モデル$p(z), p(x|z)$を明示的に定め、事後確率$p(z|x)$を未知であり、推定するものとしている。
一方で、DDPMは逆に、拡散過程（＝分類器）$q(x_{1:T}|x_0)$を明示的に定め、生成過程（＝生成器）$p(x_0, x_{1:T})$を未知であり、推定するものとしている。

一般的な潜在変数モデルと向きの違いはあれど、変分下限を最大化することに変わりはない；
生成器$p(x_0, x_{1:T})$が$\hat{p}_\theta(x_0, x_{1:T})$のように$\theta$に制御されること、また分類器は既知の事後分布$q(x_{1:T}|x_0)$を用いることに注意すると、変分下限は以下のように定義される。
$$
\theta^{\text{DDPM}} = \argmax_\theta \mathbb{E}_{x_{1:T} \sim q(x_{1:T}|x_0)} \left[ \ln{\left( \frac{\hat{p}_\theta(x_0, x_{1:T})}{q(x_{1:T}|x_0)} \right)} \right]
$$

<details><summary>変分下限の最大化はどのようなKLダイバージェンス最小化と等価であるのか？</summary><div>
上記の変分下限最大化は、数式的には以下のようなKLダイバージェンスの最小化と等価である。

$$
\theta^{\text{DDPM}} = \argmin_\theta \text{KL}(q(x_{1:T}|x_0)||\hat{p}_\theta(x_{1:T}|x_0))
$$
この式は上で述べた潜在変数モデルでの式変形を逆に辿ることで容易に得られる。

さて、式中の$\hat{p}_\theta(x_{1:T}|x_0)$は何を意味しているのか？
これは、モデルとして定義された$\hat{p}_\theta(x_0, x_{1:T})$に立脚して理論的に存在が証明された分布でしかない。
$$
\hat{p}_\theta(x_{1:T}|x_0) = \frac{\hat{p}_\theta(x_0, x_{1:T})}{p(x_0)}
$$
上式でも明らかなように、$\hat{p}_\theta(x_{1:T}|x_0)$の導出には$p(x_0)$を必要とし、当然明示的に定義されていないし、得られもしないし、また生成時に用いられもしない。
</div></details>

## DDPMの生成器・分類器
前節で述べたように、DDPMは$x_T$から$x_0$までが一列に連なったモデルである。
拡散過程$q(x_{1:T}|x_0)$では、観測変数$x_0$に対し、元のデータ成分を減衰させ、代わりにノイズを付与する操作を繰り返すことによって$x_1, \cdots, x_T$を順に得る。
$$
\begin{align*}
q(x_{1:T}|x_0) &= \prod_{t=1}^T q(x_t|x_{t-1}) \\
q(x_t|x_{t-1}) &= \mathcal{N}(x_t; \sqrt{\alpha_t}x_{t-1}, \beta_t), \quad \text{where } 1 > \alpha_0 > \alpha_1 > \cdots > \alpha_T > 0, \quad \beta_t = 1 - \alpha_t
\end{align*}
$$
第二式は、$x_t$を得るために、$x_{t-1}$に対し$\sqrt{\alpha_t}$を乗じ、代わりに分散が$\beta_t$のノイズを加算することを要請している。
$\alpha_t$は常に$1$より小さいので、拡散過程を経るにつれデータ成分がだんだん小さくなり、最終的に得られる$x_T$は$x_0$に依存しない正規分布に従うとみなせる。
$$
q(x_t|x_0) = q(x_T) = \mathcal{N}(x_T; 0, 1)
$$
一方で、生成過程$p(x_0, x_{1:T})$は、以下のような$\theta$によって制御される生成器$\hat{p}_\theta(x_0, x_{1:T})$によってモデル化される。
$$
\begin{align*}
\hat{p}_\theta(x_0, x_{1:T}) &= p(x_T) \prod_{t=1}^T p_\theta(x_{t-1}|x_t) \\
p_\theta(x_{t-1}|x_t) &= \mathcal{N} (x_{t-1}; \mu_\theta(x_t, t), \sigma_\theta^2(x_t, t)) \\
\end{align*}
$$
生成過程$p(x_0, x_{1:T})$は本来未知であり、正規分布を用いたこのモデル化に対する妥当性を評価する必要があるが、$\beta_t$が十分小さい場合は正当化されることが知られている。

拡散過程はマルコフ過程であるが、任意の時刻$t$のサンプル$x_t \sim q(x_t|x_0)$が解析的に求められるという優れた性質を持つ。
$$
\begin{align*}
q(x_t|x_0) &= \mathcal{N}(x_t; \sqrt{\bar{\alpha}_t}, \bar{\beta}_t), \quad \text{where } \bar{\alpha}_t = \prod_{\tau=1}^t \alpha_\tau, \quad \bar{\beta}_t = 1 - \bar{\alpha}_t
\end{align*}
$$
<details><summary>証明</summary><div>

証明は帰納法を用いると容易である。
1. $t=1$の時、$\bar{\alpha}_t = \alpha_1$より明らかに成立する。
2. $t=k$の時題意を満たすと仮定する。$p(x_{k+1}|x_k)$から、$x_{k+1}$は$x_k$とノイズ$\epsilon$との和に：
    $$
    x_{k+1} = \sqrt{\alpha_{k+1}} x_k + \epsilon, \quad \text{where } \epsilon \sim \mathcal{N}(0, 1-\alpha_{k+1})
    $$
    と書ける。ここで、$\sqrt{\alpha_{k+1}} x_k$が$\mathcal{N}(\sqrt{\bar{\alpha}_{k+1}}, \alpha_{k+1}(1-\bar{\alpha}_k))$に従うことに注意すれば、正規分布の再生性により、
    $$
    x_{k+1} \sim \mathcal{N}(\sqrt{\bar{\alpha}_{k+1}}, \alpha_{k+1}(1-\bar{\alpha}_k) + 1-\alpha_{k+1}) = \mathcal{N}(\sqrt{\bar{\alpha}_{k+1}}, 1-\bar{\alpha}_{k+1})
    $$
    が成立する。
</div></details>

## DDPMの学習

前々節で述べたように、DDPMは変分下限最大化によって生成過程$\hat{p}_\theta(x_0, x_{1:T})$の最適化を行う。
$$
\theta^{\text{DDPM}} = \argmax_\theta \mathbb{E}_{x_{1:T} \sim q(x_{1:T}|x_0)} \left[ \ln{\left( \frac{\hat{p}_\theta (x_0, x_{1:T})}{q(x_{1:T}|x_0)} \right)} \right]
$$

ここで、$\theta$に依存しない$x_0, x_T$を軸に変分下限を分解することを考えると、以下のような式が得られる。
$$
\begin{align*}
\theta^{\text{DDPM}}
 &= \argmax_\theta \mathbb{E}_{x_{1:T} \sim q(x_{1:T}|x_0)} \left[ -\sum_{t=2}^T \text{KL} \left( q(x_{t-1}|x_t, x_0) || \hat{p}_\theta(x_{t-1}|x_t) \right) + \ln{\hat{p}_\theta(x_0|x_1)} \right]
\end{align*}
$$

右辺の各項に基づいて、誤差$L_{t-1}$および$L_0$を
$$
\begin{align*}
L_{t-1} &= \mathbb{E}_{x_{1:T} \sim q(x_{1:T}|x_0)} \left[ \text{KL} \left( q(x_{t-1}|x_t, x_0) || \hat{p}_\theta(x_{t-1}|x_t) \right) \right] \\
L_0 &= -\mathbb{E}_{x_{1:T} \sim q(x_{1:T}|x_0)} \left[ \ln{\hat{p}_\theta(x_0|x_1)} \right]
\end{align*}
$$
と定義すると、以下のような解釈ができる：
- $L_{t-1}$を最小化することは、解析的に求められる真の事後分布$q(x_{t-1}|x_t, x_0)$と識別器（の一部）$\hat{p}_\theta(x_{t-1}|x_t)$の誤差を最小化することに等しい。
- $L_0$を最小化することは、最終的に観測データ$x_0$を再構成するステップにおける誤差（再構成誤差と呼ばれる）を最小化することに等しい。
- 両方を同時に最小化することにより、$x_T \rightarrow x_{T-1} \rightarrow \cdots \rightarrow x_1 \rightarrow x_0$の各ステップにおける誤差を最小化することができる。

<details><summary>式変形詳細</summary><div>

まず、マルコフ性に基づいて同時確率分布を分解すると、$\hat{p}_\theta(x_T)=p(x_T)$に注意して、
$$
\begin{align*}
\theta^{\text{DDPM}}
 &= \argmax_\theta \mathbb{E}_{x_{1:T} \sim q(x_{1:T}|x_0)} \left[ \ln{\left( \frac{\hat{p}_\theta(x_0|x_1)\hat{p}_\theta(x_1|x_2)\cdots, \hat{p}_\theta(x_{T-1}|x_T)\hat{p}_\theta(x_T)}{q(x_t|x_{t-1})p(x_{T-1}|x_{T-2})\cdots q(x_1|x_0)} \right)} \right] \\
 &= \argmax_\theta \mathbb{E}_{x_{1:T} \sim q(x_{1:T}|x_0)} \left[ \ln{p(x_T)} + \sum_{t=2}^T \ln{\left( \frac{\hat{p}_\theta(x_{t-1}|x_t)}{q(x_t|x_{t-1})} \right)} + \ln{\left( \frac{\hat{p}_\theta(x_0|x_1)}{q(x_1|x_0)} \right)} \right] \\
\end{align*}
$$
次に、summation中のlogarithmについて、ちょっとした小技を挟む；マルコフ性により、$q(x_t|x_{t-1}) = q(x_t|x_{t-1}, x_0)$が成立する。さらに、ベイズの定理により、
$$
q(x_t|x_{t-1}) = q(x_t|x_{t-1}, x_0) = \frac{q(x_{t-1}|x_t,x_0)q(x_t|x_0)}{q(x_{t-1}|x_0)}
$$
と言える。
これを先の式に代入して、
$$
\begin{align*}
\theta^{\text{DDPM}}
 &= \argmax_\theta \mathbb{E}_{x_{1:T} \sim q(x_{1:T}|x_0)} \left[ \ln{p(x_T)} + \sum_{t=2}^T \ln{\left( \frac{\hat{p}_\theta(x_{t-1}|x_t)}{q(x_{t-1}|x_t,x_0)} \cdot \frac{q(x_{t-1}|x_0)}{q(x_t|x_0)}\right)} + \ln{\left( \frac{\hat{p}_\theta(x_0|x_1)}{q(x_1|x_0)} \right)} \right] \\
 &= \argmax_\theta \mathbb{E}_{x_{1:T} \sim q(x_{1:T}|x_0)} \left[ \ln{p(x_T)} + \sum_{t=2}^T \ln{\left( \frac{\hat{p}_\theta(x_{t-1}|x_t)}{q(x_{t-1}|x_t,x_0)}\right)} + \sum_{t=2}^T \ln{\left(\frac{q(x_{t-1}|x_0)}{q(x_t|x_0)}\right)} + \ln{\left( \frac{\hat{p}_\theta(x_0|x_1)}{q(x_1|x_0)} \right)} \right] \\
 &= \argmax_\theta \mathbb{E}_{x_{1:T} \sim q(x_{1:T}|x_0)} \left[ \ln{p(x_T)} + \sum_{t=2}^T \ln{\left( \frac{\hat{p}_\theta(x_{t-1}|x_t)}{q(x_{t-1}|x_t,x_0)}\right)} + \ln{\left(\frac{q(x_1|x_0)}{q(x_t|x_0)}\right)} + \ln{\left( \frac{\hat{p}_\theta(x_0|x_1)}{q(x_1|x_0)} \right)} \right] \\
 &= \argmax_\theta \mathbb{E}_{x_{1:T} \sim q(x_{1:T}|x_0)} \left[ \ln{\left( \frac{p(x_T)}{q(x_t|x_0)} \right)} + \sum_{t=2}^T \ln{\left( \frac{\hat{p}_\theta(x_{t-1}|x_t)}{q(x_{t-1}|x_t,x_0)}\right)} + \ln{\hat{p}_\theta(x_0|x_1)} \right] \\
 &= \argmax_\theta \mathbb{E}_{x_{1:T} \sim q(x_{1:T}|x_0)} \left[ -\sum_{t=2}^T \text{KL} \left( q(x_{t-1}|x_t, x_0) || \hat{p}_\theta(x_{t-1}|x_t) \right) + \ln{\hat{p}_\theta(x_0|x_1)} \right] + \text{Const.}
\end{align*}
$$
が得られる。
</div></details>

<details><summary>$L_{t-1}$には$q(x_{t-1}|x_t, x_0)$が含まれているが、なぜ$p(x_{t-1}|x_t)$ではないのか？</summary><div>

DDPMは$x_{0:T}$が一列に順に連なったモデルであるため、生成過程における$x_{t-1}$のサンプリングには直前のステップ$x_t$が与えられていれば良いのではないかと考えるかもしれない。
しかし、完全に正規分布に拡散した$x_T$から、何らかの意味のある分布に従うデータを生成するプロセスを学ぶためには、「目的地」である$x_0$についての情報が不可欠である。

> 例えとして「散歩」を考える。拡散過程は$x_0$から出発して、目的地もなくフラフラ彷徨っているだけ、まさにランダムウォークである。
> 一方で、その帰り道は$x_0$を目的地としてどのように進むべきかを考えなければならない。現在地のみを元に歩いても、決して$x_0$には帰ってこれないだろう。

</div></details>

### 誤差の詳細

誤差$L_{t-1}$と$L_0$について詳細に見る。

まず$L_0$について、これは単純に正規分布を用いて表現できる。

$$
\begin{align*}
L_0 &= -\mathbb{E}_{x_{1:T} \sim q(x_{1:T}|x_0)} \left[ \ln{\mathcal{N} (x_{t-1}; \mu_\theta(x_t, t), \sigma_\theta^2(x_t, t))}  \right] \\
\end{align*}
$$

次に$L_{t-1}$について、$q(x_{t-1}|x_t, x_0)$は以下のように解析的に得られる（証明は後述）。

$$
\begin{align*}
q(x_{t-1}|x_t, x_0) &= \mathcal{N}(x_{t-1}; \tilde{\mu}_t(x_t, x_0), \tilde{\beta}_t) \\
\tilde{\mu}_t(x_t, x_0) &= \frac{\sqrt{\bar{\alpha}_{t-1}} \beta_t}{\bar{\beta}_t} x_0 + \frac{\sqrt{\alpha_t}\bar{\beta}_{t-1}}{\bar{\beta}_t} x_t \\
\tilde{\beta}_t &= \frac{\bar{\beta}_{t-1}}{\bar{\beta}_t} \beta_t \\
\end{align*}
$$

この性質によって、$L_{t-1}$は正規分布同士のKLダイバージェンスに帰着する。
またこの際、生成過程のモデルにおける分散（あるいは分散・共分散行列）$\sigma_\theta^2(x_t, t)$を、パラメータに依存しないスカラー$\sigma_t^2$で代替すると、結局KLダイバージェンスおよび$L_{t-1}$は以下のように表現できる。

$$
\begin{align*}
\text{KL} \left( q(x_{t-1}|x_t, x_0) || \hat{p}_\theta(x_{t-1}|x_t) \right)
 &= \text{KL} \left( \mathcal{N}(x_{t-1}; \tilde{\mu}_t(x_t, x_0), \tilde{\beta}_t) || \mathcal{N}(x_{t-1}; \mu_\theta(x_t, t), \sigma_\theta^2(x_t, t)) \right)\\
 &= \frac{1}{2} \left[ \ln{\frac{| \sigma_t^2 |}{| \tilde{\beta}_t |}} - d + \text{tr}{\left( {\sigma_\theta^2(x_t, t)}^{-1} \tilde{\beta}_t \right)} + {( \mu_\theta-\tilde{\mu}_t )}^\top \sigma_\theta^2 {( \mu_\theta-\tilde{\mu}_t )} \right] \\
 &= \frac{1}{2\sigma_t^2} {\left|\left| \mu_\theta-\tilde{\mu}_t \right|\right|}^2 + \text{Const.}\\
\therefore L_{t-1}
 &= \mathbb{E}_{x_{1:T} \sim q(x_{1:T}|x_0)} \left[ \frac{1}{2\sigma_t^2} {\left|\left| \mu_\theta-\tilde{\mu}_t \right|\right|}^2 \right]
\end{align*}
$$

<details><summary>証明</summary><div>

まず以下の補題の証明を行う。

補題：確率分布$p(x)$および$p(y|x)$がそれぞれ
$$
\begin{align*}
p(x) &= \mathcal{N}(x; \mu_A, \sigma_A^2) \\
p(y|x) &= \mathcal{N}(y; ax, \sigma_B^2)
\end{align*}
$$
で表現される時、事後確率$p(x|y)$について以下が成立する。
$$
\begin{align*}
p(x|y) &= \mathcal{N}(\tilde{\mu}, \tilde{\sigma}^2) \\
\frac{1}{\tilde{\sigma}} &= \frac{1}{\sigma_A^2} + \frac{a^2}{\sigma_B^2} \\
\tilde{\mu} &= \tilde{\sigma}^2 \left( \frac{\mu_A}{\sigma_A^2} + \frac{ay}{\sigma_B^2} \right)
\end{align*}
$$

<details><summary>補題の証明</summary><div>

$$
\begin{align*}
p(x|y)
 &\propto p(x) p(y|x) \\
 &\propto \exp{\left( -\frac{{|| x - \mu_A ||}^2}{2\sigma_A^2} \right)} \exp{\left( -\frac{{|| y - ax ||}^2}{2\sigma_B^2} \right)} \\
 &\propto \exp{\left( \left( -\frac{1}{2\sigma_A^2} -\frac{a^2}{2\sigma_B^2} \right) x^2 + \left( \frac{\mu_A}{\sigma_A^2} + \frac{ay}{\sigma_B^2} \right)x \right)}\\
 &\propto \exp{\left( -\frac{{|| x - \tilde{\mu} ||}^2}{2\tilde{\sigma}^2} \right)}
\end{align*}
$$

</div></details>

次に、この補題を用いて本題の証明を行う。
$q(x_{t-1}|x_t, x_0)$について、
$$
\begin{align*}
q(x_{t-1}|x_t, x_0)
 &= \frac{q(x_{t-1}, x_t | x_0)}{q(x_t | x_0)} \\
 &\propto q(x_{t-1}, x_t | x_0) \\
 &= q(x_t | x_{t-1}, x_0) q(x_{t-1} | x_0)
\end{align*}
$$
が成立する。ここで最右辺の$q(x_t | x_{t-1}, x_0)$、および$q(x_{t-1} | x_0)$について、
$$
\begin{align*}
q(x_t | x_{t-1}, x_0) &= q(x_t | x_{t-1}) = \mathcal{N}(x_t; \sqrt{\alpha_t}x_{t-1}, \beta_t) \\
q(x_{t-1} | x_0) &= \mathcal{N}(x_{t-1}; \sqrt{\bar{\alpha}_{t-1}}x_0, \bar{\beta}_{t-1})
\end{align*}
$$
が成立するため、補題を用いて、
$$
q(x_{t-1}|x_t, x_0) = \mathcal{N}(\tilde{\mu}, \tilde{\sigma}^2)
$$
$$
\begin{align*}
\frac{1}{\tilde{\sigma}^2}
 &= \frac{1}{\bar{\beta}_{t-1}} + \frac{\alpha_t}{\beta_t}\\
 &= \frac{(1 - \alpha_t) + \alpha_t (1 - \bar{\alpha}_{t-1})}{\bar{\beta}_{t-1}\beta_t} \quad (\because \beta_t = 1 - \alpha_t, \hspace{1mm} \bar{\beta}_t = 1 - \bar{\alpha}_t) \\
 &= \frac{\bar{\beta}_{t}}{\bar{\beta}_{t-1}\beta_t}\\
\tilde{\mu}
 &= \tilde{\sigma}^2 \left( \frac{\sqrt{\bar{\alpha}_{t-1}}x_0}{\bar{\beta}_{t-1}} + \frac{\sqrt{\alpha_t}x_t}{\beta_t} \right) \\
 &= \frac{\bar{\beta}_{t-1}\beta_t}{\bar{\beta}_{t}} \left( \frac{\sqrt{\bar{\alpha}_{t-1}}x_0}{\bar{\beta}_{t-1}} + \frac{\sqrt{\alpha_t}x_t}{\beta_t} \right)\\
 &= \frac{\sqrt{\bar{\alpha}_{t-1}}\beta_t}{\bar{\beta}_{t}}x_0 + \frac{\sqrt{\alpha_t}\bar{\beta}_{t-1}}{\bar{\beta}_{t}} x_t
\end{align*}
$$

</div></details>

### デノイジングスコアマッチングによる目的関数の定義